# 1 - Instalación de Dependencias
Instalamos FastAPI, Ngrok, Whisper y las herramientas de sistema necesarias (FFmpeg).

In [4]:
# 1. Instalar dependencias de Python
!pip install fastapi uvicorn pyngrok python-multipart openai-whisper requests

# 2. Instalar herramientas del sistema (FFmpeg y ZSTD para descomprimir)
# AGREGADO: 'zstd' para solucionar tu error
!sudo apt update && sudo apt install ffmpeg zstd -y

# 3. Instalar Ollama
!curl -fsSL https://ollama.com/install.sh | sh

Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
100 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as r

# verificar gpu

In [5]:
# 1. Instalar la herramienta para detectar hardware (lspci)
!sudo apt install pciutils -y

# 2. Re-ejecutar la instalación de Ollama (ahora sí detectará la GPU)
!curl -fsSL https://ollama.com/install.sh | sh

# 3. VERIFICACIÓN FINAL:
# Si esto muestra una tabla que dice "Tesla T4", ¡estás listo!
!nvidia-smi

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libpci3 pci.ids
The following NEW packages will be installed:
  libpci3 pci.ids pciutils
0 upgraded, 3 newly installed, 0 to remove and 100 not upgraded.
Need to get 343 kB of archives.
After this operation, 1,581 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 pci.ids all 0.0~2022.01.22-1ubuntu0.1 [251 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libpci3 amd64 1:3.7.0-6 [28.9 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 pciutils amd64 1:3.7.0-6 [63.6 kB]
Fetched 343 kB in 1s (316 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 3.)
debconf: falling back to frontend: Readline
debconf: u

# 2 - Iniciar Ollama y Descargar el Modelo
Aquí hay truco: Ollama funciona como un servidor, así que debemos lanzarlo en segundo plano (subprocess) antes de pedirle que descargue el modelo.

In [28]:
import subprocess
import time

# 1. Iniciar el servidor de Ollama en segundo plano
print(" Iniciando servidor Ollama...")
process = subprocess.Popen(["ollama", "serve"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
time.sleep(5)  # Esperar a que arranque

# 2. Descargar el modelo DeepSeek R1 (versión 8b optimizada)
print("⬇ Descargando DeepSeek R1 (esto puede tardar unos minutos)...")
!ollama pull deepseek-r1:8b
print(":) Modelo listo.")

 Iniciando servidor Ollama...
⬇ Descargando DeepSeek R1 (esto puede tardar unos minutos)...

:) Modelo listo.


# 3 - codigo 'main.py' dentro del colab

In [29]:
%%writefile main.py
from fastapi import FastAPI, UploadFile, File
from fastapi.middleware.cors import CORSMiddleware
import whisper
import requests
import os
import torch

app = FastAPI()

# Permitir CORS para que tu GitHub Page pueda hablar con Colab
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_methods=["*"],
    allow_headers=["*"],
)

# Cargar Whisper usando la GPU (cuda)
print("Cargando Whisper en GPU...")
model = whisper.load_model("base", device="cuda")
print("Whisper cargado.")

@app.post("/procesar-reunion")
async def procesar_reunion(file: UploadFile = File(...)):
    temp_filename = f"temp_{file.filename}"

    try:
        # 1. Guardar audio
        with open(temp_filename, "wb") as buffer:
            buffer.write(await file.read())

        # 2. Transcribir
        result = model.transcribe(temp_filename)
        transcription_text = result["text"]

        # 3. Resumir con DeepSeek (vía Ollama local en Colab)
        # DeepSeek R1 es un modelo de razonamiento, a veces piensa mucho ("<think>").
        # Le pedimos que solo nos de el resumen final.
        prompt = f"Analiza la siguiente transcripción de reunión. Genera un resumen ejecutivo en español con viñetas claras. Ignora tus pensamientos internos en la salida final.\n\nTexto: {transcription_text}"

        response = requests.post('http://localhost:11434/api/generate', json={
            "model": "deepseek-r1:8b",
            "prompt": prompt,
            "stream": False
        })

        summary = response.json()['response']

        # Limpiar etiquetas de pensamiento de DeepSeek R1 si aparecen
        if "</think>" in summary:
            summary = summary.split("</think>")[-1].strip()

        return {
            "transcription": transcription_text,
            "summary": summary
        }

    except Exception as e:
        return {"error": str(e)}

    finally:
        if os.path.exists(temp_filename):
            os.remove(temp_filename)

Overwriting main.py


In [42]:
%%writefile main.py
from fastapi import FastAPI, UploadFile, File
from fastapi.middleware.cors import CORSMiddleware
import whisper
import requests
import os
import time
import torch

app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_methods=["*"],
    allow_headers=["*"],
)

print(" Cargando Whisper (GPU)...")
model = whisper.load_model("base", device="cuda")
print(" Whisper cargado y listo.")

# Crear carpeta para guardar audios visibles
os.makedirs("grabaciones", exist_ok=True)

@app.post("/procesar-reunion")
async def procesar_reunion(file: UploadFile = File(...)):
    # Generar nombre único con hora
    filename = f"grabaciones/debug_{int(time.time())}.wav"

    print(f"\n 1. RECIBIENDO ARCHIVO...")

    try:
        # Guardar audio
        with open(filename, "wb") as buffer:
            buffer.write(await file.read())
        print(f" 2. AUDIO GUARDADO EN: {filename}")
        print("   (Busca este archivo en la carpeta de la izquierda en Colab)")

        # Transcribir
        print(" 3. INICIANDO TRANSCRIPCIÓN CON WHISPER...")
        result = model.transcribe(filename)
        transcription_text = result["text"]

        # --- DEBUG VISUAL ---
        print(f" TEXTO DETECTADO: [{transcription_text}]")

        if not transcription_text.strip():
            print(" ADVERTENCIA: Whisper no escuchó nada (texto vacío).")
            return {
                "transcription": "(Silencio o Ruido no detectado)",
                "summary": "No se detectó voz humana en el audio."
            }

        # Resumir con DeepSeek
        print(" 4. ENVIANDO A DEEPSEEK R1...")
        prompt = f"Resume esto brevemente: {transcription_text}"

        response = requests.post('http://localhost:11434/api/generate', json={
            "model": "deepseek-r1:8b",
            "prompt": prompt,
            "stream": False
        })

        summary = response.json()['response']
        # Limpieza de tags <think>
        if "</think>" in summary:
            summary = summary.split("</think>")[-1].strip()

        print(f" RESUMEN GENERADO: {summary[:50]}...") # Solo mostramos el inicio

        return {
            "transcription": transcription_text,
            "summary": summary
        }

    except Exception as e:
        print(f" ERROR CRÍTICO EN BACKEND: {str(e)}")
        return {"transcription": f"Error: {str(e)}", "summary": "Error en el servidor"}

    # ==========================================
    # ZONA DE INTERRUPTOR DE GUARDADO
    # ==========================================
    finally:
        # MODO ACTUAL: BORRAR AUDIOS (Privacidad / Ahorro de espacio)
        # Si quieres que se GUARDEN para pruebas, pon un "#" al inicio de las 2 líneas de abajo:

        if os.path.exists(filename):
             os.remove(filename)
             print(f" Limpieza automática: {filename} eliminado.")

Overwriting main.py


In [38]:
!pkill ngrok

# 4 - Exponer a NGROK

In [40]:
import subprocess
import time
import requests

print(" DIAGNÓSTICO: Revisando estado de DeepSeek/Ollama...")

# 1. Intentar conectar para ver si está vivo
try:
    response = requests.get("http://localhost:11434")
    print(" Ollama ya está corriendo.")
except:
    print(" Ollama está APAGADO. Iniciando reactivación...")

    # 2. Iniciar el servidor en segundo plano
    # Usamos nohup para que sobreviva mejor en Colab
    process = subprocess.Popen("nohup ollama serve > ollama.log 2>&1 &", shell=True)

    print(" Esperando a que Ollama despierte (10 segundos)...")
    time.sleep(10)

    # 3. Verificar de nuevo
    try:
        requests.get("http://localhost:11434")
        print(" ¡Ollama revivió exitosamente!")
    except:
        print(" ALERTA: Ollama está tardando en iniciar. Esperando 10 segundos más...")
        time.sleep(10)

# 4. Asegurar que el modelo DeepSeek está cargado en memoria
print(" Verificando que el modelo 'deepseek-r1:8b' esté listo...")
# Ejecutamos un 'pull' rápido. Si ya existe, no descargará nada, solo verificará.
!ollama pull deepseek-r1:8b

print("\n TODO LISTO. Ahora vuelve a ejecutar la celda del Servidor (la última).")

 DIAGNÓSTICO: Revisando estado de DeepSeek/Ollama...
 Ollama está APAGADO. Iniciando reactivación...
 Esperando a que Ollama despierte (10 segundos)...
 ¡Ollama revivió exitosamente!
 Verificando que el modelo 'deepseek-r1:8b' esté listo...


 TODO LISTO. Ahora vuelve a ejecutar la celda del Servidor (la última).


In [ ]:
from pyngrok import ngrok
import time

# --- PASO 1: LIMPIEZA TOTAL ---
print(" Matando procesos viejos...")
!pkill ngrok
!pkill uvicorn

# --- PASO 2: CONFIGURAR NGROK ---
NGROK_TOKEN = "38Wff95yfqrA0VesjwgaYRyctLnB_49Axs"  # <--- ¡NO OLVIDAR ESTO!
ngrok.set_auth_token(NGROK_TOKEN)

# --- PASO 3: ABRIR EL TÚNEL ---
# Abrimos el túnel en segundo plano antes de iniciar el servidor
try:
    # Si usas dominio estático, descomenta y usa esta línea:
    # public_url = ngrok.connect(8000, domain="tu-dominio.ngrok-free.app").public_url

    # Si usas dominio aleatorio:
    # public_url = ngrok.connect(8000).public_url
    # Si usas dominio estatico:
    public_url = ngrok.connect(8000, domain="br-ulises.ngrok-free.dev").public_url


    print(f"\n==================================================================")
    print(f" TU URL DE BACKEND ES:  {public_url}")
    print(f"==================================================================\n")
    print("  Copia esa URL (HTTPS) en tu index.html AHORA.")
    print("... Iniciando servidor Uvicorn... (Espera a que diga 'Application startup complete')")

    # Damos un segundo para que leas la URL
    time.sleep(2)

except Exception as e:
    print(f" Error con Ngrok: {e}")

# --- PASO 4: INICIAR SERVIDOR (MODO COMANDO) ---
# Usamos '!' para correrlo como si fuera la terminal de Linux.
# Esto evita el error "asyncio.run() cannot be called..."
!uvicorn main:app --host 0.0.0.0 --port 8000

 Matando procesos viejos...

 TU URL DE BACKEND ES:  https://joi-excludable-ulises.ngrok-free.dev

  Copia esa URL (HTTPS) en tu index.html AHORA.
... Iniciando servidor Uvicorn... (Espera a que diga 'Application startup complete')
 Cargando Whisper (GPU)...
 Whisper cargado y listo.
INFO:     Started server process [30840]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)

 1. RECIBIENDO ARCHIVO...
 2. AUDIO GUARDADO EN: grabaciones/debug_1769002578.wav
   (Busca este archivo en la carpeta de la izquierda en Colab)
 3. INICIANDO TRANSCRIPCIÓN CON WHISPER...
 TEXTO DETECTADO: [ Ahora sí debería funcionar donde voy a ser que este un bocano al teclado pero no encuentro]
 4. ENVIANDO A DEEPSEEK R1...
 RESUMEN GENERADO: Ahora sí debería funcionar donde voy a ser que **h...
INFO:     200.29.139.30:0 - "POST /procesar-reunion HTTP/1.1" 200 OK

 1. RECIBIENDO ARCHIVO...
 2. AUDIO GUARDADO 